CAA 2023-09-08 0103AM

# Setting up the requirements:

1. Cloning the repository from: https://github.com/ultralytics/yolov5
2. Mounting and getting files ready


In [1]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo

# Change the current directory to the YOLOv5 repository and install dependencies
%cd yolov5
!pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 15967, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 15967 (delta 43), reused 51 (delta 24), pack-reused 15880
Receiving objects: 100% (15967/15967), 14.64 MiB | 26.26 MiB/s, done.
Resolving deltas: 100% (10942/10942), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.4/616.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [2]:
# set up environment
import os
os.environ["DATASET_DIRECTORY"] = "/content/datasets"
os.environ['QT_QPA_PLATFORM'] = 'offscreen'
import cv2

In [3]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
print(os.getcwd())

/content/yolov5


In [5]:
# Copy dataset over and extract zip
!cp "/content/drive/MyDrive/MDP/Task 1 pics.v4i.yolov5pytorch.zip" "task1.zip"

In [6]:
# Copy new YAML in
!cp "/content/drive/MyDrive/MDP/custom-hyp.yaml" "custom-hyp.yaml"

In [7]:
!unzip -q task1.zip

# Train Our Custom YOLOv5 model
Here, we are able to pass a number of arguments:

img: define input image size
batch: determine batch size
epochs: define the number of training epochs. (Note: often, 3000+ are common here!)
data: Our dataset locaiton is saved in the dataset.location
weights: specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
cache: cache images for faster training
hyp: hyperparameters to use

In [8]:
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.0.1+cu118 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


In [9]:
!cd data/

In [10]:
!ls data/hyps/

hyp.no-augmentation.yaml  hyp.scratch-high.yaml  hyp.scratch-med.yaml
hyp.Objects365.yaml	  hyp.scratch-low.yaml	 hyp.VOC.yaml


In [11]:
cat custom-hyp.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
# Hyperparameters for high-augmentation COCO training from scratch
# python train.py --batch 32 --cfg yolov5m6.yaml --weights '' --data coco.yaml --img 1280 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # fo

In [12]:
#!python train.py --img 640 --epochs 50 --batch 32 --data data.yaml --weights yolov5s.pt --cache --hyp custom-hyp.yaml
!python train.py --img 416 --epochs 50 --batch 16 --data data.yaml --weights yolov5s.pt --cache --hyp custom-hyp.yaml

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=custom-hyp.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-218-g9e97ac3 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, transl

In [13]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Run inference test

In [ ]:
!python detect.py --weights /content/drive/MyDrive/MDP/task1-shunyao-v1.pt --conf 0.1 --source /content/drive/MyDrive/MDP/20230830_163027.jpg

detect: weights=['/content/drive/MyDrive/MDP/task1-shunyao-v1.pt'], source=/content/drive/MyDrive/MDP/20230830_163027.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-215-ga6659d0 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 
Model summary: 157 layers, 7093732 parameters, 0 gradients, 16.0 GFLOPs
image 1/1 /content/drive/MyDrive/MDP/20230830_163027.jpg: 640x288 1 11, 1 38, 1 39, 2 99s, 90.9ms
Speed: 0.6ms pre-process, 90.9ms inference, 107.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp4


In [14]:
# DISPLAY THE RESULT
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the image
image_path = 'runs/detect/exp/20230830_163027.jpg'
img = mpimg.imread(image_path)

# Display the image
plt.figure(figsize=(18, 18))  # Optional: Set the figure size
plt.imshow(img)
plt.axis('off')  # Optional: Hide the axis
plt.show()

FileNotFoundError: ignored

In [ ]:
#export img
from google.colab import files
files.download('runs/detect/exp5/20230830_163027.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>